In [347]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import json

In [348]:
load_dotenv()
key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

gpt3_model = "gpt-3.5-turbo-0125"
gpt4_model = "gpt-4-0125-preview"

In [349]:
def call_openai(conversation_history, output_json=False, functions=None, model=gpt3_model, temperature=1):
    response_format = {"type": "text"}
    tools = []
    if output_json:
        response_format["type"] = "json_object"
    if functions:
        for function in functions:
            tools.append({
                "type": "function",
                "function": function
            })
    if tools:
        return client.chat.completions.create(model=model, response_format=response_format, messages=conversation_history, tools=tools, temperature=temperature)
    return client.chat.completions.create(model=model, response_format=response_format, messages=conversation_history, temperature=temperature)

# Website Generation

In [350]:
website_json_frame = {
    "header": {
        "title": "This should be title of the web site. Kind of logo. Should be short",
        "image_search_prompt": "short two word description of brand logo"
    },
    "intro": {
        "heading": "this should be heading for page",
        "description": "this should be 2-3 line description about the business",
        "button_label": "Label text of button"
    },
    "custom_links": {
        "links": [
            {
                "button_label": "Label for the button",
                "button_icon": "One of the 12 icons: (need to type all icons here)"
            }
        ]
    },
    "services_section": {
        "heading": "",
        "button_label": "Each service will have a button on ui and this label will be used for all services, For ex: Book this Service/Book",
        "currency": "currency unit to use one of USD, EUR, GBP",
        "services": [
            {
                "name": "name of the service",
                "description": "service desc",
                "price": "integer value of price in unit defined above",
                "image_search_prompt": "short two word description for above service"
            }
        ]
    },
    "text": {
        "heading": "",
        "description": "",
        "button_label": ""
    },
    "gallery_section": {
        "heading": "",
        "button_label": "",
        "images": [
            {
                "image_description": "",
                "image_search_prompt": "short two word descrioption of above image"

            }
        ]
    },
    "product_section": {
        "heading": "",
        "main_button_label": "Label for main button show to right of Heading, For example: Shope all",
        "button_label": "Each product will have a button on ui and this label will be used for all products, For ex: Buy/Buy Now",
        "currency": "currency unit to use one of USD, EUR, GBP",
        "products": [
            {
                "name": "",
                "price": "integer value of price in unit defined above",
                "image_search_prompt": "short two word description for above product"
            }
        ]
    },
    "testimonial_section": {
        "heading": "Headin of the testimonial section",
        "testimonials": [
            {
                "quote": "Reviw given by user. Should be 3-5 lines.",
                "written_by": "name of a person",
                "designation": "",
                "gender": "male or female"
            }
        ]
    },
    "contact_form": {
        "heading": "heading of the contact form"
    },
    "social_links": {
        "links": [
            {
                "type": "facebook/linkedin/twitter/tiktok",
                "link": ""
            }
        ]
    }
}

In [351]:
bussiness_description = """
Company Name: Code With Luv
Description: I sell competitive programming courses
"""

website_generate_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant which is being used a website and its content, you will be provided with a json format, each field in the\
              json is to be filled properly in accordance with bussiness details given by the user and the description of each and every field described in format"
    },
    {
        "role": "user",
        "content": f"Here is the json format for the website{website_json_frame}. You are free to drop any of the section if you dont think it is usefull in this \
            website and generate content for the sections you will should be there. A website should generally have one of products or services section depending on\
             wether the business sells services or products Here is the businnes description given by user{bussiness_description}. \
            Generate the json for the website according to given format"
    }
]
response = call_openai(
    conversation_history=website_generate_messages, output_json=True)

In [352]:
website_json = json.loads(response.choices[0].message.content)
website_json

{'header': {'title': 'Code With Luv',
  'image_search_prompt': 'coding education'},
 'intro': {'heading': 'Master Competitive Programming with Code With Luv!',
  'description': 'Enhance your coding skills and excel in competitive programming with our expertly crafted courses.',
  'button_label': 'Explore Courses'},
 'services_section': {'heading': 'Our Courses',
  'button_label': 'Book this Course',
  'currency': 'USD',
  'services': [{'name': 'Basic Programming Fundamentals',
    'description': 'Learn the basics of programming languages and problem-solving techniques.',
    'price': 99,
    'image_search_prompt': 'coding basics'},
   {'name': 'Advanced Algorithm Design',
    'description': 'Master advanced algorithms and data structures to excel in competitive programming.',
    'price': 149,
    'image_search_prompt': 'algorithm design'}]},
 'testimonial_section': {'heading': 'Student Reviews',
  'testimonials': [{'quote': 'Code With Luv helped me improve my problem-solving skills si

# Chat section

In [353]:
website_chat_messages = [
    {
        "role": "system",
        "content": f"You are a helpful assistant which is being used in a site builder app, you help user in building a beautiful and informational site for their business, \
give them suggestions to improve certain section's content if asked(one section at a time). Based on this json format: {website_json_frame} , \
this is the current website: {website_json}, user can ask you to remove certain sections, add certain sections or modify certains sections in json. \
The changes you can make or only limited to what can be added/removed/modified in json, If user askes to add any new ui component or something like that, \
you should refuse to user in simple language and not mention things like json etc as real user is unaware of that. Your replies should be concise. You should also not call any function in such cases as well"
    }
]
old_website_json = website_json
new_website_json = website_json

In [354]:
website_chat_messages

[{'role': 'system',
  'content': "You are a helpful assistant which is being used in a site builder app, you help user in building a beautiful and informational site for their business, give them suggestions to improve certain section's content if asked(one section at a time). Based on this json format: {'header': {'title': 'This should be title of the web site. Kind of logo. Should be short', 'image_search_prompt': 'short two word description of brand logo'}, 'intro': {'heading': 'this should be heading for page', 'description': 'this should be 2-3 line description about the business', 'button_label': 'Label text of button'}, 'custom_links': {'links': [{'button_label': 'Label for the button', 'button_icon': 'One of the 12 icons: (need to type all icons here)'}]}, 'services_section': {'heading': '', 'button_label': 'Each service will have a button on ui and this label will be used for all services, For ex: Book this Service/Book', 'currency': 'currency unit to use one of USD, EUR, GBP'

In [355]:
functions = [
    {
        "name": "change_website",
        "description": "This function should be called when user explicitly asks to change the website content. This should not be called when asked for suggestions or for ui changes as this can only change content",
        "parameters": {
            "type": "object",
            "properties": {
                "changes_to_made": {
                    "type": "string",
                    "description": "A prompt which describes what changes should be made to the website",
                }
            },
            "required": ["changes_to_made"],
        },
    }
]

In [380]:
prompt = "Add a nav bar to header section"
website_chat_messages.append({
    "role": "user",
    "content": prompt
})

In [381]:
website_chat_messages

[{'role': 'system',
  'content': "You are a helpful assistant which is being used in a site builder app, you help user in building a beautiful and informational site for their business, give them suggestions to improve certain section's content if asked(one section at a time). Based on this json format: {'header': {'title': 'This should be title of the web site. Kind of logo. Should be short', 'image_search_prompt': 'short two word description of brand logo'}, 'intro': {'heading': 'this should be heading for page', 'description': 'this should be 2-3 line description about the business', 'button_label': 'Label text of button'}, 'custom_links': {'links': [{'button_label': 'Label for the button', 'button_icon': 'One of the 12 icons: (need to type all icons here)'}]}, 'services_section': {'heading': '', 'button_label': 'Each service will have a button on ui and this label will be used for all services, For ex: Book this Service/Book', 'currency': 'currency unit to use one of USD, EUR, GBP'

In [384]:
response = call_openai(conversation_history=website_chat_messages, functions=functions, model=gpt4_model)

In [385]:
response

ChatCompletion(id='chatcmpl-99CWHTAVmTXlIg8LZHVjlUCX5PnKC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I can help with content modifications within the structures we've discussed, but adding new UI components like a navigation bar isn't something I can do directly. For enhancements like adding a navigation bar, it's typically handled within the site's design or theme settings through your website builder or by editing the site's HTML/CSS directly. I recommend looking into those options to add a navigation bar to your website.", role='assistant', function_call=None, tool_calls=None))], created=1711980105, model='gpt-4-0125-preview', object='chat.completion', system_fingerprint='fp_a7daf7c51e', usage=CompletionUsage(completion_tokens=81, prompt_tokens=1927, total_tokens=2008))

In [376]:
if response.choices[0].finish_reason == 'tool_calls':
    website_chat_messages.append(response.choices[0].message)
    tool_call = response.choices[0].message.tool_calls[0]
    argument = json.loads(response.choices[0].message.tool_calls[0].function.arguments)['changes_to_made']
    print(f'Argument: {argument}')
    modify_conversation_agent = [
        {
            "role": "system",
            "content": f"You are a helpful assistant which is being used in a site builder app, you help user in building a beautiful and informational site for their business, \
            give them suggestions to improve it and then call appropriate functions to make changes to the website. Based on this json format: {website_json_frame} , \
            this is the current website: {new_website_json}, user can ask you to remove certain sections, add certain sections or modify certains sections, and you perform \
            those action in accordance with original json fomrat given."
        },
        {
            "role": "user",
            "content": f"{argument}. Output the modified website json"
        }
    ]
    modified_website_response = call_openai(conversation_history=modify_conversation_agent, output_json=True)
    old_website_json = new_website_json
    new_website_json = json.loads(modified_website_response.choices[0].message.content)
    website_chat_messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": tool_call.function.name,
            "content": f"The website has been modified to {new_website_json}"
        }
    )
    response = call_openai(conversation_history=website_chat_messages, functions=functions)
    website_chat_messages.append(
        {
            "role": "assistant",
            "content": response.choices[0].message.content
        }
    )
else:
    website_chat_messages.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })

Argument: Add a gallery section with relevant images


In [377]:
website_chat_messages

[{'role': 'system',
  'content': "You are a helpful assistant which is being used in a site builder app, you help user in building a beautiful and informational site for their business, give them suggestions to improve certain section's content if asked(one section at a time). Based on this json format: {'header': {'title': 'This should be title of the web site. Kind of logo. Should be short', 'image_search_prompt': 'short two word description of brand logo'}, 'intro': {'heading': 'this should be heading for page', 'description': 'this should be 2-3 line description about the business', 'button_label': 'Label text of button'}, 'custom_links': {'links': [{'button_label': 'Label for the button', 'button_icon': 'One of the 12 icons: (need to type all icons here)'}]}, 'services_section': {'heading': '', 'button_label': 'Each service will have a button on ui and this label will be used for all services, For ex: Book this Service/Book', 'currency': 'currency unit to use one of USD, EUR, GBP'

In [378]:
new_website_json

{'header': {'title': 'Code With Luv',
  'image_search_prompt': 'coding education'},
 'intro': {'heading': 'Master Competitive Programming with Code With Luv!',
  'description': 'Enhance your coding skills and excel in competitive programming with our expertly crafted courses.',
  'button_label': 'Explore Courses'},
 'services_section': {'heading': 'Our Courses',
  'button_label': 'Book this Course',
  'currency': 'USD',
  'services': [{'name': 'Basic Programming Fundamentals',
    'description': 'Learn the basics of programming languages and problem-solving techniques.',
    'price': 99,
    'image_search_prompt': 'coding basics'},
   {'name': 'Advanced Algorithm Design',
    'description': 'Master advanced algorithms and data structures to excel in competitive programming.',
    'price': 149,
    'image_search_prompt': 'algorithm design'}]},
 'testimonial_section': {'heading': 'Student Reviews',
  'testimonials': [{'quote': 'Code With Luv helped me improve my problem-solving skills si

In [379]:
old_website_json

{'header': {'title': 'Code With Luv',
  'image_search_prompt': 'coding education'},
 'intro': {'heading': 'Master Competitive Programming with Code With Luv!',
  'description': 'Enhance your coding skills and excel in competitive programming with our expertly crafted courses.',
  'button_label': 'Explore Courses'},
 'services_section': {'heading': 'Our Courses',
  'button_label': 'Book this Course',
  'currency': 'USD',
  'services': [{'name': 'Basic Programming Fundamentals',
    'description': 'Learn the basics of programming languages and problem-solving techniques.',
    'price': 99,
    'image_search_prompt': 'coding basics'},
   {'name': 'Advanced Algorithm Design',
    'description': 'Master advanced algorithms and data structures to excel in competitive programming.',
    'price': 149,
    'image_search_prompt': 'algorithm design'}]},
 'testimonial_section': {'heading': 'Student Reviews',
  'testimonials': [{'quote': 'Code With Luv helped me improve my problem-solving skills si